## Datacleaning

In [ ]:
import pandas as pd
import re

# 데이터셋 불러오기
# train_df = pd.read_csv('../data/full_train_fin.csv')
# train_df = pd.read_csv('../data/train.csv')
# dev_df = pd.read_csv('../data/dev.csv')
# test_df = pd.read_csv('../data/test.csv')

# 정규표현식을 사용하여 텍스트를 정리하는 함수 정의
def clean_text(text):
    text = re.sub(r'"', '', text)  # 큰따옴표 " 제거
    text = re.sub(r'\s{2,}', ' ', text)  # 2칸 이상의 공백을 1칸으로 변경
    text = re.sub(r'\s\.', '.', text)  # 공백 + 점(.)을 점(.)으로 변경
    text = re.sub(r'\s,', ',', text)  # 공백 + 쉼표(,)를 쉼표(,)로 변경
    text = re.sub(r'!!', '!', text)  # !!를 !로 변경
    text = re.sub(r'\?\?', '?', text)  # ??를 ?로 변경
    text = re.sub(r'\s!', '!', text)  # 공백 + 느낌표(!)를 느낌표(!)로 변경
    text = re.sub(r'\s\?', '?', text)  # 공백 + 물음표(?)를 물음표(?)로 변경
    text = re.sub(r'\s;', ';', text)  # 공백 + 세미콜론(;)을 세미콜론(;)으로 변경
    text = re.sub(r';', '.', text)  # 세미콜론(;)을 점(.)으로 변경
    text = re.sub(r'[ㄱ-ㅎ&&[^ㅋ]]', '', text)  # ㄱ~ㅎ 중 ㅋ를 제외한 모든 자음 삭제
    
    # 작은 따옴표 안의 텍스트를 제외하고 모음만 있는 경우 삭제
    text = re.sub(r"(?<!')([ㅏ-ㅣ])(?!')", '', text)  # 작은 따옴표로 감싸지 않은 모음만 삭제
    return text




In [ ]:
# 정리 함수(dialogue와 summary 열에 적용)
full_df_cleaned['dialogue'] = full_df_cleaned['dialogue'].apply(clean_text)
full_df_cleaned['summary'] = full_df_cleaned['summary'].apply(clean_text)
full_df_cleaned['topic'] = full_df_cleaned['topic'].apply(clean_text)



In [ ]:
# 정리된 데이터를 새로운 CSV 파일로 저장
full_df_cleaned.to_csv('../data/cleaned_train_fin_2.csv', index=False)

In [3]:
%pip install koeda

  Obtaining dependency information for koeda from https://files.pythonhosted.org/packages/db/4c/cb7daad97e1e2a06b082cea42e93eb106374e439fd2c0e7d817fd0ff5ef3/koeda-0.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for konlpy>=0.5.2 from https://files.pythonhosted.org/packages/5a/95/896914d9461c12f07e6f8afb1f7462e28395fc46b54fcbb96f1ea3cff8fb/konlpy-0.6.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tweepy==3.10.0 from https://files.pythonhosted.org/packages/67/c3/6bed87f3b1e5ed2f34bd58bf7978e308c86e255193916be76e5a5ce5dfca/tweepy-3.10.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for requests-oauthlib>=0.7.0 from https://files.pythonhosted.org/packages/3b/5d/63d4ae3b9daea098d5d6f5da83984853c1bbacd5dc826764b249fe119d24/requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for JPype1>=0.7.0 from https://files.pythonhosted.org/packages/74/98/d6517002355b0585d0e66f7b0283c7f6e2271c898a886e1ebac09

In [1]:
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.


## EDA

In [1]:
import pandas as pd
from koeda import EasyDataAugmentation
import re

# EDA 적용 함수
def augment_text_data_with_EDA(text, repetition=1):
    """입력된 문장에 대해서 EDA를 통해 데이터 증강"""
    eda = EasyDataAugmentation(morpheme_analyzer="Okt")
    result = eda(text, p=(0.3, 0.3, 0.3, 0.3), repetition=repetition)
    return result

# 화자 태그와 대화 분리 함수
def split_speaker_and_text(dialogue):
    """화자 태그와 대화 내용을 분리"""
    pattern = r'(#Person\d+#): (.+)'
    lines = dialogue.split('\n')
    split_lines = [re.match(pattern, line) for line in lines if re.match(pattern, line)]
    return [(match.group(1), match.group(2)) for match in split_lines]

# 화자 태그와 대화 결합 함수
def join_speaker_and_text(speaker_text_pairs):
    """화자 태그와 대화 내용을 다시 결합"""
    return '\n'.join([f'{speaker}: {text}' for speaker, text in speaker_text_pairs])

# CSV 파일 불러오기
data_path = '../data/'
df = pd.read_csv(f'{data_path}cleaned_train.csv')
# train_df = pd.read_csv(f'{data_path}cleaned_train.csv')
# val_df = pd.read_csv(f'{data_path}cleaned_dev.csv')
# df = pd.concat([train_df,val_df],axis=0)

# 필요한 열만 가져오기
#df_filtered = train_df[['fname', 'dialogue', 'summary', 'topic']]
df_filtered = df[['fname', 'dialogue', 'summary', 'topic']]
#df_filtered = df_filtered[:200]

# 증강된 데이터를 담을 리스트
augmented_rows = []

# dialogue 열에 대해 EDA 적용
for index, row in df_filtered.iterrows():
    original_text = row['dialogue']
    
    # 화자 태그와 대화 내용 분리
    speaker_text_pairs = split_speaker_and_text(original_text)
    
    # 대화 내용만 추출하여 EDA 적용
    augmented_text_lists = [augment_text_data_with_EDA(text, repetition=3) for speaker, text in speaker_text_pairs]

    # EDA 결과와 화자 태그 결합
    for augmented_texts in zip(*augmented_text_lists):  # 증강된 각 문장 조합을 함께 결합
        new_speaker_text_pairs = [(speaker_text_pairs[i][0], augmented_text) for i, augmented_text in enumerate(augmented_texts)]
        augmented_dialogue = join_speaker_and_text(new_speaker_text_pairs)

        # 증강된 문장들과 원래 데이터를 결합
        new_row = {
            'fname': row['fname'],
            'dialogue': augmented_dialogue,
            'summary': row['summary'],
            'topic': row['topic']
        }
        augmented_rows.append(new_row)

    # 100개마다 진행 상황 출력
    if index % 100 == 0:
        print(f"Processed {index+1} rows out of {len(df_filtered)}")

# 리스트를 DataFrame으로 변환
augmented_data = pd.DataFrame(augmented_rows)

# 결과 출력 또는 저장
#augmented_data.to_csv(f'{data_path}augmented_train.csv', index=False)


Processed 1 rows out of 12457
Processed 101 rows out of 12457
Processed 201 rows out of 12457
Processed 301 rows out of 12457
Processed 401 rows out of 12457
Processed 501 rows out of 12457
Processed 601 rows out of 12457
Processed 701 rows out of 12457
Processed 801 rows out of 12457
Processed 901 rows out of 12457
Processed 1001 rows out of 12457
Processed 1101 rows out of 12457
Processed 1201 rows out of 12457
Processed 1301 rows out of 12457
Processed 1401 rows out of 12457
Processed 1501 rows out of 12457
Processed 1601 rows out of 12457
Processed 1701 rows out of 12457
Processed 1801 rows out of 12457
Processed 1901 rows out of 12457
Processed 2001 rows out of 12457
Processed 2101 rows out of 12457
Processed 2201 rows out of 12457
Processed 2301 rows out of 12457
Processed 2401 rows out of 12457
Processed 2501 rows out of 12457
Processed 2601 rows out of 12457
Processed 2701 rows out of 12457
Processed 2801 rows out of 12457
Processed 2901 rows out of 12457
Processed 3001 rows ou

In [2]:
augmented_data

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 ?\n...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_0,"#Person1#: 안녕하세요,스미스 씨.왜는저 ? 의사입니다. 오늘 오셨나요호킨스...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
2,train_0,"#Person1#: 안녕하세요, 기적 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 ...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
3,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요 여성 아주머니 ?\n...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
4,train_1,"#Person1#: 안녕하세요, , 어떻게 ?\n#Person2#: 안녕하세요, 피...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
...,...,...,...,...
37348,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일\n#Person2#: 음...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
37349,train_12458,#Person1#: 오늘 행복해 ? 않아.좀있어 일 무슨보이지\n#Person2#:...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
37350,train_12459,#Person1#: 엄마 에 이 가족을 방문하기 비행기를 타고 갈 거예요 오늘 가방...,#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
37351,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 항공기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [3]:
full_df = pd.concat([df, augmented_data],axis=0)


In [4]:
# dialogue 열에서 중복된 행 제거 (첫 번째만 남기고 나머지 제거)
full_df_cleaned = full_df.drop_duplicates(subset=['dialogue'], keep='first')

# 중복 제거 후 결과 확인
print(f"중복 제거 후 데이터 크기: {full_df_cleaned.shape}")

#인덱스 리셋
full_df_cleaned.reset_index(drop=True, inplace=True)

# fname 열을 'train_0', 'train_1', ..., 'train_46420'으로 변환
full_df_cleaned['fname'] = [f"train_{i}" for i in range(len(full_df_cleaned))]

# # 중복이 제거된 데이터 저장 (원하는 경로로 변경)
# output_path = '../data/full_train_fin.csv'
# full_df_cleaned_2.to_csv(output_path, index=False)

print("중복이 제거된 CSV 파일이 성공적으로 저장되었습니다.")


중복 제거 후 데이터 크기: (49765, 4)
중복이 제거된 CSV 파일이 성공적으로 저장되었습니다.


/tmp/ipykernel_36679/4217223280.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df_cleaned['fname'] = [f"train_{i}" for i in range(len(full_df_cleaned))]


In [5]:
import pandas as pd
import re

# 데이터셋 불러오기
# train_df = pd.read_csv('../data/full_train_fin.csv')
# train_df = pd.read_csv('../data/train.csv')
# dev_df = pd.read_csv('../data/dev.csv')
# test_df = pd.read_csv('../data/test.csv')

# 정규표현식을 사용하여 텍스트를 정리하는 함수 정의
def clean_text(text):
    text = re.sub(r'"', '', text)  # 큰따옴표 " 제거
    text = re.sub(r'\s{2,}', ' ', text)  # 2칸 이상의 공백을 1칸으로 변경
    text = re.sub(r'\s\.', '.', text)  # 공백 + 점(.)을 점(.)으로 변경
    text = re.sub(r'\s,', ',', text)  # 공백 + 쉼표(,)를 쉼표(,)로 변경
    text = re.sub(r'!!', '!', text)  # !!를 !로 변경
    text = re.sub(r'\?\?', '?', text)  # ??를 ?로 변경
    text = re.sub(r'\s!', '!', text)  # 공백 + 느낌표(!)를 느낌표(!)로 변경
    text = re.sub(r'\s\?', '?', text)  # 공백 + 물음표(?)를 물음표(?)로 변경
    text = re.sub(r'\s;', ';', text)  # 공백 + 세미콜론(;)을 세미콜론(;)으로 변경
    text = re.sub(r';', '.', text)  # 세미콜론(;)을 점(.)으로 변경
    text = re.sub(r'[ㄱ-ㅎ&&[^ㅋ]]', '', text)  # ㄱ~ㅎ 중 ㅋ를 제외한 모든 자음 삭제
    
    # 작은 따옴표 안의 텍스트를 제외하고 모음만 있는 경우 삭제
    text = re.sub(r"(?<!')([ㅏ-ㅣ])(?!')", '', text)  # 작은 따옴표로 감싸지 않은 모음만 삭제
    return text




In [6]:
# 정리 함수(dialogue와 summary 열에 적용)
full_df_cleaned['dialogue'] = full_df_cleaned['dialogue'].apply(clean_text)
full_df_cleaned['summary'] = full_df_cleaned['summary'].apply(clean_text)
full_df_cleaned['topic'] = full_df_cleaned['topic'].apply(clean_text)



/tmp/ipykernel_36679/3202723221.py:22: FutureWarning: Possible set intersection at position 4
  text = re.sub(r'[ㄱ-ㅎ&&[^ㅋ]]', '', text)  # ㄱ~ㅎ 중 ㅋ를 제외한 모든 자음 삭제


/tmp/ipykernel_36679/1875915690.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df_cleaned['dialogue'] = full_df_cleaned['dialogue'].apply(clean_text)
/tmp/ipykernel_36679/1875915690.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df_cleaned['summary'] = full_df_cleaned['summary'].apply(clean_text)
/tmp/ipykernel_36679/1875915690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [7]:
# 정리된 데이터를 새로운 CSV 파일로 저장
full_df_cleaned.to_csv('../data/train_eda.csv', index=False)

In [8]:
# train_df = pd.read_csv('../data/train.csv')
# dev_df = pd.read_csv('../data/dev.csv')
# df = pd.concat([train_df,dev_df], axis=0)

In [9]:
df

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스
...,...,...,...,...
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직


In [10]:
# # dialogue 열에서 중복된 행 제거 (첫 번째만 남기고 나머지 제거)
# # full_df_cleaned = full_df.drop_duplicates(subset=['dialogue'], keep='first')

# # 중복 제거 후 결과 확인
# # print(f"중복 제거 후 데이터 크기: {full_df_cleaned.shape}")

# #인덱스 리셋
# df.reset_index(drop=True, inplace=True)

# # fname 열을 'train_0', 'train_1', ..., 'train_46420'으로 변환
# df['fname'] = [f"train_{i}" for i in range(len(df))]

# # # 중복이 제거된 데이터 저장 (원하는 경로로 변경)
# output_path = '../data/full_train.csv'
# df.to_csv(output_path, index=False)

# print("중복이 제거된 CSV 파일이 성공적으로 저장되었습니다.")
